<a href="https://colab.research.google.com/github/aasiro/NLP-classification/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Считаем сайт с релевантном странички.
Если честно не я не смогла разобраться, как оптимальнее вытащить более 100 данных со страницы, поэтому разбила их на части: с 1 по 4 апреля, с 5 по 12, 13-16б17-23, 24-30 апреля.

In [ ]:
import requests

url = 'http://www.cbr.ru/rbr/insidedko/'
params = {'UniDbQuery.Posted':True,
'UniDbQuery.SPhrase':"",
'UniDbQuery.From':"01.04.2021",
'UniDbQuery.To':"04.04.2021",
'UniDbQuery.TypeOfMsg':"-1",
'UniDbQuery.RegionSelect':"-1"
}
  

r = requests.get(url, params=params)

In [ ]:
from bs4 import BeautifulSoup as bs

начнем парсить сайт с результатами 1 - 4 апреля

In [ ]:
soup = bs(r.text, 'html.parser')


Проверяем, что действительно данные загружаются на определенные даты

In [ ]:
new = soup.find_all('div', class_ = "datepicker-filter")
new


[<div class="datepicker-filter" data-datepicker-period-filter="" data-default-value-from="29.04.2021" data-default-value-to="13.05.2021" data-max-date="13.05.2021" data-min-date="24.05.2018">
 <input class="datepicker-filter_input-from" name="UniDbQuery.From" type="hidden" value="01.04.2021"/>
 <input class="datepicker-filter_input-to" name="UniDbQuery.To" type="hidden" value="04.04.2021"/>
 <button class="datepicker-filter_button" type="button">01.04.2021 - 04.04.2021</button>
 <div class="datepicker-filter_modal">
 <div class="datepicker-filter_head">
 <div class="datepicker-filter_close"></div>
 </div>
 <div class="datepicker-filter_tabs">
 <div class="datepicker-filter_tab-from"></div> - 
                 <div class="datepicker-filter_tab-to"></div>
 </div>
 <div class="datepicker-filter_body">
 <div class="datepicker-filter_datepicker-from datepicker-custom"></div>
 <div class="datepicker-filter_datepicker-to datepicker-custom"></div>
 </div>
 <div class="datepicker-filter_footer"

In [ ]:
quant = soup.find_all('div', class_ = "results")
quant

[<div class="results" xmlns:xsltblock="urn:XsltBlock">
 <div class="results_top">
 <div class="results_counter">Найдено документов: 47</div>
 </div>
 </div>]

Очевидно, что за этот период времени мы находим 47 записей, поэтому за эту выгрузку должно получиться 47 словарей.

In [ ]:
results = soup.find_all('div', class_ = "title-source offset-md-4")
results

[<div class="title-source offset-md-4"><a href="doc?id=22434" style="display:none">АО им. Ленина, ОГРН 1026101716354, ИНН 6137000266</a><div class="title">АО им. Ленина, ОГРН 1026101716354, ИНН 6137000266</div>
 <div class="subtitle">Банк России 02.04.2021 принял решение о приостановлении эмиссии обыкновенных акций Акционерного общества им. Ленина (Ростовская область), размещенных путем закрытой подписки, регистрационный номер дополнительного выпуска 1-01-57652-P-001D.</div>
 <div class="source">Решение в отношении ценных бумаг</div>
 <div class="source">Южное главное управление Центрального банка Российской Федерации</div>
 </div>,
 <div class="title-source offset-md-4"><a href="doc?id=22433" style="display:none">АО «Константа Инвестмент Групп», ОГРН 1187746065869, ИНН 9710047352</a><div class="title">АО «Константа Инвестмент Групп», ОГРН 1187746065869, ИНН 9710047352</div>
 <div class="subtitle">Банк России 02.04.2021 принял решение об отказе в государственной регистрации выпуска при

Так как эмитент на сайте записан таким же классом, что и решение, выделим его сначала в отдельный словарь, а затем прочитаем оставшиеся поля и объединим 2 словаря в один

In [ ]:
source = []
for i in results:
  source.append(i.find_all('div', class_='source'))

issuer = []

for i in source:
  new = str(i[1])
  issuer.append({'issuer':new[20:-6]})

print(issuer[1:7])  



[{'issuer': 'Главное управление Центрального банка Российской Федерации по Центральному федеральному округу г. Москва'}, {'issuer': 'Главное управление Центрального банка Российской Федерации по Центральному федеральному округу г. Москва'}, {'issuer': 'Главное управление Центрального банка Российской Федерации по Центральному федеральному округу г. Москва'}, {'issuer': 'Главное управление Центрального банка Российской Федерации по Центральному федеральному округу г. Москва'}, {'issuer': 'Главное управление Центрального банка Российской Федерации по Центральному федеральному округу г. Москва'}, {'issuer': 'Волго-Вятское главное управление Центрального банка Российской Федерации'}]


In [ ]:
dec = []
for item in results:
  dec.append({'title': item.find('div',class_="title").get_text(strip=True),
              'desc':item.find('div',class_="subtitle").get_text(strip=True),
              'result':item.find('div', class_="source").get_text(strip=True)})
print(dec[:2])  

[{'title': 'АО им. Ленина, ОГРН 1026101716354, ИНН 6137000266', 'desc': 'Банк России 02.04.2021 принял решение о приостановлении эмиссии обыкновенных акций Акционерного общества им. Ленина (Ростовская область), размещенных путем закрытой подписки, регистрационный номер дополнительного выпуска 1-01-57652-P-001D.', 'result': 'Решение в отношении ценных бумаг'}, {'title': 'АО «Константа Инвестмент Групп», ОГРН 1187746065869, ИНН 9710047352', 'desc': 'Банк России 02.04.2021 принял решение об отказе в государственной регистрации выпуска привилегированных акций Акционерного общества «Константа Инвестмент Групп» (г. Москва), размещаемых путем закрытой подписки.', 'result': 'Решение в отношении ценных бумаг'}]


In [ ]:
for i in range(len(dec)):
  dec[i].update(issuer[i])

print(dec[:2])

[{'title': 'АО им. Ленина, ОГРН 1026101716354, ИНН 6137000266', 'desc': 'Банк России 02.04.2021 принял решение о приостановлении эмиссии обыкновенных акций Акционерного общества им. Ленина (Ростовская область), размещенных путем закрытой подписки, регистрационный номер дополнительного выпуска 1-01-57652-P-001D.', 'result': 'Решение в отношении ценных бумаг', 'issuer': 'Южное главное управление Центрального банка Российской Федерации'}, {'title': 'АО «Константа Инвестмент Групп», ОГРН 1187746065869, ИНН 9710047352', 'desc': 'Банк России 02.04.2021 принял решение об отказе в государственной регистрации выпуска привилегированных акций Акционерного общества «Константа Инвестмент Групп» (г. Москва), размещаемых путем закрытой подписки.', 'result': 'Решение в отношении ценных бумаг', 'issuer': 'Главное управление Центрального банка Российской Федерации по Центральному федеральному округу г. Москва'}]


сохраним первые 47 наблюдений в csv

In [ ]:
import csv

In [ ]:
with open('file.csv', 'w', newline='') as file:
  writer = csv.writer(file, delimiter=';')
  writer.writerow(['title','desc','result','issuer'])
  for item in dec:
    writer.writerow([item['title'],item['desc'],item['result'],item['issuer']])


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_table('file.csv', sep = ';', header=0)
df.shape

(47, 4)

# Добавление в файл остальных дат (код аналогичный, кроме имененных дат)

In [ ]:
url = 'http://www.cbr.ru/rbr/insidedko/'
params = {'UniDbQuery.Posted':True,
'UniDbQuery.SPhrase':"",
'UniDbQuery.From':"05.04.2021",
'UniDbQuery.To':"12.04.2021",
'UniDbQuery.TypeOfMsg':"-1",
'UniDbQuery.RegionSelect':"-1"
}
  

r = requests.get(url, params=params)
soup = bs(r.text, 'html.parser')

In [ ]:
results = soup.find_all('div', class_ = "title-source offset-md-4")
source = []
for i in results:
  source.append(i.find_all('div', class_='source'))

issuer = []

for i in source:
  new = str(i[1])
  issuer.append({'issuer':new[20:-6]})

dec = []
for item in results:
  dec.append({'title': item.find('div',class_="title").get_text(strip=True),
              'desc':item.find('div',class_="subtitle").get_text(strip=True),
              'result':item.find('div', class_="source").get_text(strip=True)})
  
for i in range(len(dec)):
  dec[i].update(issuer[i])

print(len(dec))

90


In [ ]:
with open('file.csv', 'a', newline='') as file:
  writer = csv.writer(file, delimiter=';')
  for item in dec:
    writer.writerow([item['title'],item['desc'],item['result'],item['issuer']])

  

In [ ]:
import pandas as pd

In [ ]:
url = 'http://www.cbr.ru/rbr/insidedko/'
params = {'UniDbQuery.Posted':True,
'UniDbQuery.SPhrase':"",
'UniDbQuery.From':"13.04.2021",
'UniDbQuery.To':"16.04.2021",
'UniDbQuery.TypeOfMsg':"-1",
'UniDbQuery.RegionSelect':"-1"
}
  

r = requests.get(url, params=params)
soup = bs(r.text, 'html.parser')

In [ ]:
results = soup.find_all('div', class_ = "title-source offset-md-4")
source = []
for i in results:
  source.append(i.find_all('div', class_='source'))

issuer = []

for i in source:
  new = str(i[1])
  issuer.append({'issuer':new[20:-6]})

dec = []
for item in results:
  dec.append({'title': item.find('div',class_="title").get_text(strip=True),
              'desc':item.find('div',class_="subtitle").get_text(strip=True),
              'result':item.find('div', class_="source").get_text(strip=True)})
  
for i in range(len(dec)):
  dec[i].update(issuer[i])

print(len(dec))

80


In [ ]:
with open('file.csv', 'a', newline='') as file:
  writer = csv.writer(file, delimiter=';')
  for item in dec:
    writer.writerow([item['title'],item['desc'],item['result'],item['issuer']])

In [ ]:
df = pd.read_table('file.csv', sep = ';', header = 0)
df.shape

(217, 4)

In [ ]:
url = 'http://www.cbr.ru/rbr/insidedko/'
params = {'UniDbQuery.Posted':True,
'UniDbQuery.SPhrase':"",
'UniDbQuery.From':"17.04.2021",
'UniDbQuery.To':"23.04.2021",
'UniDbQuery.TypeOfMsg':"-1",
'UniDbQuery.RegionSelect':"-1"
}
  

r = requests.get(url, params=params)
soup = bs(r.text, 'html.parser')

In [ ]:
results = soup.find_all('div', class_ = "title-source offset-md-4")
source = []
for i in results:
  source.append(i.find_all('div', class_='source'))

issuer = []

for i in source:
  new = str(i[1])
  issuer.append({'issuer':new[20:-6]})

dec = []
for item in results:
  dec.append({'title': item.find('div',class_="title").get_text(strip=True),
              'desc':item.find('div',class_="subtitle").get_text(strip=True),
              'result':item.find('div', class_="source").get_text(strip=True)})
  
for i in range(len(dec)):
  dec[i].update(issuer[i])

print(len(dec))

84


In [ ]:
with open('file.csv', 'a', newline='') as file:
  writer = csv.writer(file, delimiter=';')
  for item in dec:
    writer.writerow([item['title'],item['desc'],item['result'],item['issuer']])

In [ ]:
url = 'http://www.cbr.ru/rbr/insidedko/'
params = {'UniDbQuery.Posted':True,
'UniDbQuery.SPhrase':"",
'UniDbQuery.From':"24.04.2021",
'UniDbQuery.To':"30.04.2021",
'UniDbQuery.TypeOfMsg':"-1",
'UniDbQuery.RegionSelect':"-1"
}
  

r = requests.get(url, params=params)
soup = bs(r.text, 'html.parser')

In [ ]:
results = soup.find_all('div', class_ = "title-source offset-md-4")
source = []
for i in results:
  source.append(i.find_all('div', class_='source'))

issuer = []

for i in source:
  new = str(i[1])
  issuer.append({'issuer':new[20:-6]})

dec = []
for item in results:
  dec.append({'title': item.find('div',class_="title").get_text(strip=True),
              'desc':item.find('div',class_="subtitle").get_text(strip=True),
              'result':item.find('div', class_="source").get_text(strip=True)})
  
for i in range(len(dec)):
  dec[i].update(issuer[i])

print(len(dec))

98


In [ ]:
with open('file.csv', 'a', newline='') as file:
  writer = csv.writer(file, delimiter=';')
  for item in dec:
    writer.writerow([item['title'],item['desc'],item['result'],item['issuer']])

# Обработка данных из полученного файла

In [ ]:
df.head(10)

,title,desc,result,issuer
0,"АО им. Ленина, ОГРН 1026101716354, ИНН 6137000266",Банк России 02.04.2021 принял решение о приост...,Решение в отношении ценных бумаг,Южное главное управление Центрального банка Ро...
1,"АО «Константа Инвестмент Групп», ОГРН 11877460...",Банк России 02.04.2021 принял решение об отказ...,Решение в отношении ценных бумаг,Главное управление Центрального банка Российск...
2,"АО «Константа Инвестмент Групп», ОГРН 11877460...",Банк России 02.04.2021 принял решение об отказ...,Решение в отношении ценных бумаг,Главное управление Центрального банка Российск...
3,"АО «Импакт Капитал», ОГРН 1197746387761, ИНН 7...",Банк России 01.04.2021 принял решение об отказ...,Решение в отношении ценных бумаг,Главное управление Центрального банка Российск...
4,"АО «Горизонты будущего», ОГРН 1217700017633, И...",Банк России 02.04.2021 принял решение о госуда...,Решение в отношении ценных бумаг,Главное управление Центрального банка Российск...
5,"АО «Альтернатива-Инвест», ОГРН 1217700017765, ...",Банк России 02.04.2021 принял решение о госуда...,Решение в отношении ценных бумаг,Главное управление Центрального банка Российск...
6,"АО «Технопарк ВТ», ОГРН 1095837001446, ИНН 583...",Банк России 02.04.2021 принял решение о госуда...,Решение в отношении ценных бумаг,Волго-Вятское главное управление Центрального ...
7,"АО «Первый канал», ОГРН 1027700222330, ИНН 771...",Банк России 02.04.2021 принял решение о госуда...,Решение в отношении ценных бумаг,Главное управление Центрального банка Российск...
8,"АО «КБК «Черёмушки», ОГРН 1037728017580, ИНН 7...",Банк России 02.04.2020 принял решение о госуда...,Решение в отношении ценных бумаг,Главное управление Центрального банка Российск...
9,"АО «ФПГ ЭНЕРГОКОНТРАКТ», ОГРН 1027739479404, И...",Банк России от 02.04.2021 принял решение об ан...,Решение в отношении ценных бумаг,Главное управление Центрального банка Российск...


In [ ]:
df.shape

(399, 4)

**Таким образом, мы получили 399 записей на сайте ЦБ относительно решений за апрель 2021.**

**Проверим реальное количество типов решений ЦБ относительно** 

In [ ]:
df['result'].value_counts()

Решение в отношении ценных бумаг                    196
Решение в отношении иных финансовых инструментов     21
Name: result, dtype: int64

**Так как получается всего 2 класса, то создадим бинарную переменную, релевантную этим решениям. Пусть решения в отношении ценных бумаг будет 1, а в отношении иных финансовых инструментов - 0**

In [ ]:
def num(results):
  if results == 'Решение в отношении ценных бумаг':
    return 1
  else:
    return 0

In [ ]:
df['decision'] = df.apply(lambda row: num(row['result']), axis=1)

In [ ]:
df['decision'].value_counts()

1    196
0     21
Name: decision, dtype: int64

In [ ]:
y = df['decision'].values

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

**Сделаем первичную токенизацию слов. В качестве зависимой переменной решено сипользовать колонку title, так как в ней очевидны различия по решению и будет легко проверить**

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
tokenizer = nltk.tokenize.WordPunctTokenizer()

stemmer = nltk.stem.WordNetLemmatizer()

naming = []
for i in df['title']:
  word = tokenizer.tokenize(i)
  naming.append(word)

words = []
for line in naming:
  stam = ' '.join(stemmer.lemmatize(token) for token in line)
  stam = stam.split(' ')
  new = []
  words.append(new)
  for i in stam:
    if i.isalpha():
      new.append(i)
      

words[390:]


      



   
  



[]

In [ ]:
pip install pymorphy2

     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 8.2MB 29.0MB/s 


In [ ]:
pip install pymorphy2-dicts

     |████████████████████████████████| 7.1MB 17.7MB/s 


In [ ]:
pip install DAWG-Python

**Проведем лемматизацию зависимой переменной с помощью бибилиотеки для лематизации русского языка. Так как текст не самыйс сложный, можно обратиться к этой библиотеке, принимая во внимание некоторые допущения**

In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [ ]:
nominative = []
for line in words:
  title =[]
  nominative.append(title)
  for i in line:
      word = morph.parse(i.lower())
      if word[0].normal_form == 'инна':
        title.append(word[2].normal_form)
      else:
        title.append(word[0].normal_form)

nominative[2]      

['ао', 'константа', 'инвестмент', 'группа', 'огрн', 'инн']

In [ ]:
nominative[:21]

[['ао', 'они', 'ленин', 'огрн', 'инн'],
 ['ао', 'константа', 'инвестмент', 'группа', 'огрн', 'инн'],
 ['ао', 'константа', 'инвестмент', 'группа', 'огрн', 'инн'],
 ['ао', 'импакт', 'капитал', 'огрн', 'инн'],
 ['ао', 'горизонт', 'будущее', 'огрн', 'инн'],
 ['ао', 'альтернатива', 'инвест', 'огрн', 'инн'],
 ['ао', 'технопарк', 'вт', 'огрн', 'инн'],
 ['ао', 'первый', 'канал', 'огрн', 'инн'],
 ['ао', 'кбк', 'черёмушки', 'огрн', 'инн'],
 ['ао', 'фпг', 'энергоконтракт', 'огрн', 'инн'],
 ['ао', 'пбк', 'огрн', 'инн'],
 ['ао', 'по', 'керчьрыбпром', 'огрн', 'инн'],
 ['ао', 'ряжский', 'арза', 'огрн', 'инн'],
 ['ао', 'поэз', 'ульяновск', 'огрн', 'инн'],
 ['ао', 'гдк', 'берелех', 'огрн', 'инн'],
 ['ао', 'усмк', 'огрн', 'инн', 'не', 'присвоить'],
 ['ао', 'винсадский', 'огрн', 'инн'],
 ['пао', 'сегежа', 'группа', 'огрн', 'инн'],
 ['ао', 'корпорация', 'развитие', 'московский', 'область', 'огрн', 'инн'],
 ['ао', 'джетта', 'огрн', 'не', 'присвоить', 'инн', 'не', 'присвоить'],
 ['о',
  'отказ',
  'в',
  'р

**На мой взгляд, стоп-словами в данном случае можно назвать все элементы списка, длина которых = 1 или предлоги(чаще всего), поэтому избавимся от них. другие слова значимые.**

In [ ]:
nomin = []
for line in nominative:
  length = []
  nomin.append(length)
  for i in line:
    if len(i) > 1:
      length.append(i)

nomin[20]     


['отказ',
 'регистрация',
 'изменение',
 'дополнение',
 'правило',
 'доверительный',
 'управление',
 'паевой',
 'инвестиционный',
 'фонд',
 'представить',
 'ооо',
 'ук',
 'альфа',
 'капитал']

In [ ]:
name = []
for line in nomin:
  a = ' '.join(token for token in line)
  name.append(a)

print(name[:20])

['ао они ленин огрн инн', 'ао константа инвестмент группа огрн инн', 'ао константа инвестмент группа огрн инн', 'ао импакт капитал огрн инн', 'ао горизонт будущее огрн инн', 'ао альтернатива инвест огрн инн', 'ао технопарк вт огрн инн', 'ао первый канал огрн инн', 'ао кбк черёмушки огрн инн', 'ао фпг энергоконтракт огрн инн', 'ао пбк огрн инн', 'ао по керчьрыбпром огрн инн', 'ао ряжский арза огрн инн', 'ао поэз ульяновск огрн инн', 'ао гдк берелех огрн инн', 'ао усмк огрн инн не присвоить', 'ао винсадский огрн инн', 'пао сегежа группа огрн инн', 'ао корпорация развитие московский область огрн инн', 'ао джетта огрн не присвоить инн не присвоить']


**Посчитаем биграммный TF-IDF**

In [ ]:
tfidf = TfidfVectorizer( min_df = 5,max_df=0.4, ngram_range=(1, 2))

feature = tfidf.fit_transform(name)


table = pd.DataFrame(
    feature.todense(),
    columns=tfidf.get_feature_names())

In [ ]:
table.head(20)

,банк,банк втб,втб,втб пао,группа,группа огрн,доверительный,доверительный управление,дополнение,дополнение правило,зао,изменение,изменение дополнение,инвестиционный,инвестиционный фонд,инн не,капитал,капитал огрн,не,не присвоить,огрн не,ооо,ооо сфо,ооо ук,паевой,паевой инвестиционный,пао,пао огрн,правило,правило доверительный,представить,представить ооо,присвоить,присвоить инн,регистрация,регистрация изменение,регистрация правило,сфо,ук,управление,управление паевой,фонд,фонд представить
0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.683887,0.729588,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.683887,0.729588,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.624929,0.780682,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
print(table.columns.tolist())

['банк', 'банк втб', 'втб', 'втб пао', 'группа', 'группа огрн', 'доверительный', 'доверительный управление', 'дополнение', 'дополнение правило', 'зао', 'изменение', 'изменение дополнение', 'инвестиционный', 'инвестиционный фонд', 'инн не', 'капитал', 'капитал огрн', 'не', 'не присвоить', 'огрн не', 'ооо', 'ооо сфо', 'ооо ук', 'паевой', 'паевой инвестиционный', 'пао', 'пао огрн', 'правило', 'правило доверительный', 'представить', 'представить ооо', 'присвоить', 'присвоить инн', 'регистрация', 'регистрация изменение', 'регистрация правило', 'сфо', 'ук', 'управление', 'управление паевой', 'фонд', 'фонд представить']


**Начнем реализовывать bag of words.
Посчитаем количество слов в преобразованной до именительного падежа переменной - nomin**

In [ ]:
h = dict()
for line in nomin:
    for token in line:
        h[token] = h.get(token, 0) + 1
for key in h.keys():
    print(key + '\t' + str(h[key]))

ао	162
они	1
ленин	1
огрн	196
инн	196
константа	2
инвестмент	3
группа	7
импакт	2
капитал	14
горизонт	1
будущее	1
альтернатива	1
инвест	3
технопарк	1
вт	1
первый	2
канал	1
кбк	1
черёмушки	1
фпг	1
энергоконтракт	1
пбк	1
по	2
керчьрыбпром	1
ряжский	1
арза	1
поэз	1
ульяновск	1
гдк	1
берелех	1
усмк	1
не	36
присвоить	36
винсадский	1
пао	23
сегежа	1
корпорация	4
развитие	2
московский	1
область	2
джетта	1
отказ	1
регистрация	21
изменение	16
дополнение	16
правило	21
доверительный	21
управление	23
паевой	21
инвестиционный	22
фонд	23
представить	21
ооо	23
ук	18
альфа	2
вкусвилл	1
оркк	1
роствертол	1
стс	1
сопф	1
инфраструктурный	1
облигация	1
россеть	2
кубань	1
мф	1
диджитал	1
сфо	5
азимут	2
энергомашбанк	1
ск	3
ренессанс	1
здоровье	1
радиозавод	1
башкиравтодор	1
амза	1
вентпром	1
кургансельмаш	1
арсагер	2
афм	1
открытие	1
тинькофф	1
губернский	1
яхта	1
клуб	1
коматечь	1
водоканал	1
дзержинскхлеб	1
рацин	1
окб	1
мэи	1
розтех	1
мпк	1
нииэфа	1
октекс	1
банк	9
втб	7
фцнивт	1
снпо	1
элерон	1
локосерв

In [ ]:
len(h)

322

**Создадим словарь с наиболее часто встречающимися словами**

In [ ]:
import heapq
most_freq = heapq.nlargest(50, h, key=h.get)

In [ ]:
sentence_vectors = []
for sentence in nomin:
    sent_vec = []
    for token in most_freq:
        if token in sentence:
            sent_vec.append(1)
        else:
            sent_vec.append(0)
    sentence_vectors.append(sent_vec)

**переведем полученный мешок слов в векторное отображение**

In [ ]:
import numpy as np
X = np.asarray(sentence_vectors)

**Посмотрим первый классификатор. Так как задача достаточно тривиальна: 2 класса и маленький датасет, попробуем реализовать логрегрессию с регуляризатором l1, который будет штрафовать излишне хорошую модель**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, stratify=y)

In [ ]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

173 44 173 44


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer 

In [ ]:

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

def plot_scores(optimizer):
    scores = [[item[0]['C'], 
               item[1], 
               (np.sum((item[2]-item[1])**2)/(item[2].size-1))**0.5] for item in optimizer.grid_scores_]
    scores = np.array(scores)
    plt.semilogx(scores[:,0], scores[:,1])
    plt.fill_between(scores[:,0], scores[:,1]-scores[:,2], 
                                  scores[:,1]+scores[:,2], alpha=0.3)
    plt.show()

In [ ]:
lr = LogisticRegression(max_iter = 4000, solver='liblinear', penalty= 'l1')
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=4000,
                   multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_train, lr.predict(X_train)))
print(accuracy_score(y_test, lr.predict(X_test)))

1.0
1.0


In [ ]:
y_train.mean(), y_test.mean()

(0.9017341040462428, 0.9090909090909091)

In [ ]:
print(roc_auc_score(y_train, lr.predict_proba(X_train)[:, 1]))
print(roc_auc_score(y_test, lr.predict_proba(X_test)[:, 1]))

1.0
1.0


In [ ]:
grid_searcher = GridSearchCV(
    LogisticRegression(max_iter=100, solver='liblinear', random_state=13),
    param_grid={
        'C': [0.01, 0.05, 0.1, 0.5, 1, 10],
        'penalty': ['l1']
    },
    cv=3,
    scoring='roc_auc'
)

In [ ]:
grid_searcher.fit(X_train, y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=13, solver='liblinear',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.01, 0.05, 0.1, 0.5, 1, 10], 'penalty': ['l1']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='roc_auc', verbose=0)

In [ ]:
print(roc_auc_score(y_train, grid_searcher.predict_proba(X_train)[:, 1]))
print(roc_auc_score(y_test, grid_searcher.predict_proba(X_test)[:, 1]))

1.0
1.0


In [ ]:
pd.DataFrame(grid_searcher.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_penalty,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002649,0.002645,0.001205,0.000234,0.01,l1,"{'C': 0.01, 'penalty': 'l1'}",0.5,0.5,0.5,0.5,0.0,6
1,0.000780,0.000010,0.000986,0.000006,0.05,l1,"{'C': 0.05, 'penalty': 'l1'}",1.0,1.0,1.0,1.0,0.0,1
2,0.000737,0.000023,0.000971,0.000003,0.1,l1,"{'C': 0.1, 'penalty': 'l1'}",1.0,1.0,1.0,1.0,0.0,1
3,0.000810,0.000036,0.000996,0.000020,0.5,l1,"{'C': 0.5, 'penalty': 'l1'}",1.0,1.0,1.0,1.0,0.0,1
4,0.000800,0.000036,0.001159,0.000133,1,l1,"{'C': 1, 'penalty': 'l1'}",1.0,1.0,1.0,1.0,0.0,1
5,0.000850,0.000018,0.001063,0.000042,10,l1,"{'C': 10, 'penalty': 'l1'}",1.0,1.0,1.0,1.0,0.0,1


In [ ]:
grid_searcher.best_params_

{'C': 0.05, 'penalty': 'l1'}

In [ ]:
grid_searcher.best_score_


1.0

In [ ]:
lr = grid_searcher.best_estimator_
lr

LogisticRegression(C=0.05, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=13, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

**Модель логрегрессии переобучена, даже с вводимыми штрафами**

**Так как анализ с помощью bag of words показывает переобученнуб модель, даже с учетом тяжелого регуляризатора, попробуем построить модель с данным из TF-IDF сначала для биграммных значений**

In [ ]:
newidf = pd.DataFrame()
for i in table.columns:
  if ' ' in i:
    newidf[i] = table[i]


newidf.head(5)    

,банк втб,втб пао,группа огрн,доверительный управление,дополнение правило,изменение дополнение,инвестиционный фонд,инн не,капитал огрн,не присвоить,огрн не,ооо сфо,ооо ук,паевой инвестиционный,пао огрн,правило доверительный,представить ооо,присвоить инн,регистрация изменение,регистрация правило,управление паевой,фонд представить
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.729588,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.729588,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.780682,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
newidf.shape

(217, 22)

In [ ]:
X_idf = newidf[:].values

Попробуем для наглядности изменить размер тестовой выборки на 0.8 от всего датасета.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_idf,y, test_size = 0.8, stratify=y)

In [ ]:
print(len(x_train), len(x_test), len(y_train), len(y_test))

43 174 43 174


In [ ]:
lr = LogisticRegression(max_iter = 4000, solver='liblinear', penalty= 'l1')
lr.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=4000,
                   multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
print(accuracy_score(y_train, lr.predict(x_train)))
print(accuracy_score(y_test, lr.predict(x_test)))

0.9069767441860465
0.9022988505747126


**В данном случае видно, что модель работает хуже, чем единица, но все равно достаточно хорошо, чтобы прогнозировать**

**так как дальнейшие преобразования бессмыслены, раз модель вновь переобучена. попробуем посмотрить на комбинации би и триграммной модели tf-idf**

In [ ]:
tfidf = TfidfVectorizer( min_df = 5,max_df=0.4, ngram_range=(1, 3))

feature = tfidf.fit_transform(name)


table = pd.DataFrame(
    feature.todense(),
    columns=tfidf.get_feature_names())

In [ ]:
newidf2 = pd.DataFrame()
for i in table.columns:
  if ' ' in i:
    newidf2[i] = table[i]


newidf2.head(5)    

,банк втб,банк втб пао,втб пао,втб пао огрн,группа огрн,группа огрн инн,доверительный управление,доверительный управление паевой,дополнение правило,дополнение правило доверительный,изменение дополнение,изменение дополнение правило,инвестиционный фонд,инвестиционный фонд представить,инн не,инн не присвоить,капитал огрн,капитал огрн инн,не присвоить,не присвоить инн,огрн инн не,огрн не,огрн не присвоить,ооо сфо,ооо ук,паевой инвестиционный,паевой инвестиционный фонд,пао огрн,пао огрн инн,правило доверительный,правило доверительный управление,представить ооо,представить ооо ук,присвоить инн,присвоить инн не,регистрация изменение,регистрация изменение дополнение,регистрация правило,регистрация правило доверительный,управление паевой,управление паевой инвестиционный,фонд представить,фонд представить ооо
0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.589394,0.589394,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.589394,0.589394,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.615366,0.615366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X_idf2 = newidf2[:].values

In [ ]:
z_train, z_test, y_train, y_test = train_test_split(X_idf2,y, test_size = 0.2, stratify=y)

In [ ]:
lr = LogisticRegression(max_iter = 4000, solver='liblinear', penalty= 'l1')
lr.fit(z_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=4000,
                   multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
print(accuracy_score(y_train, lr.predict(z_train)))
print(accuracy_score(y_test, lr.predict(z_test)))

1.0
1.0


In [ ]:
print(roc_auc_score(y_train, lr.predict_proba(z_train)[:, 1]))
print(roc_auc_score(y_test, lr.predict_proba(z_test)[:, 1]))

1.0
1.0


**Очевидно, что модель все возможные классификаторы сильно подстариваются под существующие данные. Могу предположить, что подобная картина возникла из-за маленького датасета и слишком очевидных различий при выведении заключения.**

**Как уже было замечено выше при изменени пропорции тестовой и обучающей выборки, классификтор начинает работать чуть хуже, но тем не менее все равно показывает отличные результаты**

# Группировка по эмитентам

In [ ]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>title</th>\n      <th>desc</th>\n      <th>result</th>\n      <th>issuer</th>\n      <th>decision</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>АО им. Ленина, ОГРН 1026101716354, ИНН 6137000266</td>\n      <td>Банк России 02.04.2021 принял решение о приостановлении эмиссии обыкновенных акций Акционерного общества им. Ленина (Ростовская область), размещенных путем закрытой подписки, регистрационный номер дополнительного выпуска 1-01-57652-P-001D.</td>\n      <td>Решение в отношении ценных бумаг</td>\n      <td>Южное главное управление Центрального банка Российской Федерации</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>АО «Константа Инвестмент Групп», ОГРН 1187746065869, ИНН 9710047352</td>\n      <td>Банк России 02.04.2021 принял решение об отказе в государственной регистрации выпуска привилегированных акций Акционе

In [ ]:
df['issuer'].unique()

array(['Южное главное управление Центрального банка Российской Федерации',
       'Главное управление Центрального банка Российской Федерации по Центральному федеральному округу г. Москва',
       'Волго-Вятское главное управление Центрального банка Российской Федерации',
       'Отделение по Республике Крым Южного главного управления Центрального банка Российской Федерации',
       'Отделение по Орловской области Главного управления Центрального банка Российской Федерации по Центральному федеральному округу',
       'Дальневосточное главное управление Центрального банка Российской Федерации',
       'Уральское главное управление Центрального банка Российской Федерации',
       'Департамент корпоративных отношений',
       'Северо-Западное главное управление Центрального банка Российской Федерации',
       'Сибирское главное управление Центрального банка Российской Федерации'],
      dtype=object)

In [ ]:
df[df['issuer']=='Южное главное управление Центрального банка Российской Федерации' ].to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>title</th>\n      <th>desc</th>\n      <th>result</th>\n      <th>issuer</th>\n      <th>decision</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>АО им. Ленина, ОГРН 1026101716354, ИНН 6137000266</td>\n      <td>Банк России 02.04.2021 принял решение о приостановлении эмиссии обыкновенных акций Акционерного общества им. Ленина (Ростовская область), размещенных путем закрытой подписки, регистрационный номер дополнительного выпуска 1-01-57652-P-001D.</td>\n      <td>Решение в отношении ценных бумаг</td>\n      <td>Южное главное управление Центрального банка Российской Федерации</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>16</th>\n      <td>АО "Винсадское", ОГРН 1022600962152, ИНН 2618003921</td>\n      <td>Банк России 01.04.2021 принял решение об аннулировании индивидуального кода 001D дополнительного выпуска привилегированных акций тип

In [ ]:
df[df['issuer']=='Главное управление Центрального банка Российской Федерации по Центральному федеральному округу г. Москва'].to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>title</th>\n      <th>desc</th>\n      <th>result</th>\n      <th>issuer</th>\n      <th>decision</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>1</th>\n      <td>АО «Константа Инвестмент Групп», ОГРН 1187746065869, ИНН 9710047352</td>\n      <td>Банк России 02.04.2021 принял решение об отказе в государственной регистрации выпуска привилегированных акций Акционерного общества «Константа Инвестмент Групп» (г. Москва), размещаемых путем закрытой подписки.</td>\n      <td>Решение в отношении ценных бумаг</td>\n      <td>Главное управление Центрального банка Российской Федерации по Центральному федеральному округу г. Москва</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>АО «Константа Инвестмент Групп», ОГРН 1187746065869, ИНН 9710047352</td>\n      <td>Банк России 02.04.2021 принял решение об отказе в государственной регистрации дополнител

In [ ]:
df[df['issuer']=='Волго-Вятское главное управление Центрального банка Российской Федерации'].to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>title</th>\n      <th>desc</th>\n      <th>result</th>\n      <th>issuer</th>\n      <th>decision</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>6</th>\n      <td>АО «Технопарк ВТ», ОГРН 1095837001446, ИНН 5837040720</td>\n      <td>Банк России 02.04.2021 принял решение о государственной регистрации дополнительного выпуска акций обыкновенных акционерного общества «Технопарк высоких технологий» (Пензенская область), размещаемых путем закрытой подписки. Дополнительному выпуску ценных бумаг присвоен  регистрационный номер 1-02-04469-E-004D.</td>\n      <td>Решение в отношении ценных бумаг</td>\n      <td>Волго-Вятское главное управление Центрального банка Российской Федерации</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>13</th>\n      <td>АО «ПОЭЗ «Ульяновск», ОГРН 1147328004174, ИНН 7328080049</td>\n      <td>Банк России 01.04.2021 принял решение о госу

In [ ]:
df[df['issuer']=='Отделение по Республике Крым Южного главного управления Центрального банка Российской Федерации'].to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>title</th>\n      <th>desc</th>\n      <th>result</th>\n      <th>issuer</th>\n      <th>decision</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>11</th>\n      <td>АО "ПО "КЕРЧЬРЫБПРОМ", ОГРН 1159102115270, ИНН 9111018553</td>\n      <td>Банк России 02.04.2021 принял решение о государственной регистрации отчета об итогах выпуска акций обыкновенных акционерного общества "Производственное объединение "КЕРЧЬРЫБПРОМ" (Республика Крым), размещенных путем конвертации в них акций той же категории (типа), номинальная стоимость которых выражена в иностранной валюте, регистрационный номер выпуска 1-01-16687-А.</td>\n      <td>Решение в отношении ценных бумаг</td>\n      <td>Отделение по Республике Крым Южного главного управления Центрального банка Российской Федерации</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>257</th>\n      <td>ПАО «СЕВАСТОПОЛЬСКОЕ АВТОТРАН

In [ ]:
df[df['issuer']=='Отделение по Орловской области Главного управления Центрального банка Российской Федерации по Центральному федеральному округу'].to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>title</th>\n      <th>desc</th>\n      <th>result</th>\n      <th>issuer</th>\n      <th>decision</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>12</th>\n      <td>АО "Ряжский АРЗ", ОГРН 1026200662531, ИНН 6214000915</td>\n      <td>Банк России 02.04.2021 принял решение об отказе в государственной регистрации отчета об итогах дополнительного выпуска акций обыкновенных Акционерного общества "Ряжский авторемонтный завод" (Рязанская область), размещавшихся путем закрытой подписки, регистрационный номер дополнительного выпуска 1-01-12035-А-001D, дата государственной регистрации 09.09.2020, признании указанного дополнительного выпуска ценных бумаг несостоявшимся и аннулировании его государственной регистрации.</td>\n      <td>Решение в отношении ценных бумаг</td>\n      <td>Отделение по Орловской области Главного управления Центрального банка Российской Федерации по

In [ ]:
df[df['issuer']=='Дальневосточное главное управление Центрального банка Российской Федерации'].to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>title</th>\n      <th>desc</th>\n      <th>result</th>\n      <th>issuer</th>\n      <th>decision</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>14</th>\n      <td>АО "ГДК "Берелех", ОГРН 1024900949919, ИНН 4905006253</td>\n      <td>Банк России 02.04.2021 принял решение о приостановлении эмиссии акций обыкновенных акционерного общества "Горно-добывающая компания "Берелех" (Магаданская область), размещаемых путем закрытой подписки.</td>\n      <td>Решение в отношении ценных бумаг</td>\n      <td>Дальневосточное главное управление Центрального банка Российской Федерации</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>71</th>\n      <td>АО "Аэропорт Якутск", ОГРН 1101435011710, ИНН 1435235578</td>\n      <td>Банк России 09.04.2021 принял решение о государственной регистрации изменений в документ, содержащий условия размещения акций обыкновенных акционерног

In [ ]:
df[df['issuer']=='Уральское главное управление Центрального банка Российской Федерации'].to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>title</th>\n      <th>desc</th>\n      <th>result</th>\n      <th>issuer</th>\n      <th>decision</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>15</th>\n      <td>АО "УСМК" ОГРН, ИНН не присвоены</td>\n      <td>Банк России 01.04.2021 принял решение о государственной регистрации выпуска обыкновенных акций акционерного общества «Урало-Сибирская Металлургическая Компания», (Челябинская область), размещаемых путем обмена на акции акционерного общества, создаваемого в результате преобразования, долей участников в уставном капитале реорганизуемого в форме преобразования общества с ограниченной ответственностью «Урало-Сибирская Металлургическая Компания».Выпуску ценных бумаг присвоен регистрационный номер 1-01-12925-К.Решение о государственной регистрации указанного выпуска ценных бумаг вступает в силу с даты государственной регистрации акционерного общества «Урало-

In [ ]:
df[df['issuer']=='Департамент корпоративных отношений'].to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>title</th>\n      <th>desc</th>\n      <th>result</th>\n      <th>issuer</th>\n      <th>decision</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>17</th>\n      <td>ПАО «Сегежа Групп», ОГРН 1207700498279, ИНН 9703024202</td>\n      <td>Банк России 01.04.2021 принял решение о государственной регистрации дополнительного выпуска и регистрации проспекта обыкновенных акций публичного акционерного общества Группы компаний «Сегежа» (г. Москва), размещаемых путем открытой подписки. Дополнительному выпуску ценных бумаг присвоен регистрационный номер 1-01-87154-Н.Решения о государственной регистрации дополнительного выпуска акций и о регистрации проспекта акций при приобретении обществом публичного статуса вступают в силу со дня внесения в единый государственный реестр юридических лиц сведений о фирменном наименовании общества, содержащем указание на то, что общество явля

In [ ]:
df[df['issuer']=='Северо-Западное главное управление Центрального банка Российской Федерации'].to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>title</th>\n      <th>desc</th>\n      <th>result</th>\n      <th>issuer</th>\n      <th>decision</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>46</th>\n      <td>АО «НИИЭФА», ОГРН 1137847503067, ИНН 7817331468</td>\n      <td>Банк России 31.03.2021 принял решение о государственной регистрации отчета об итогах дополнительного выпуска обыкновенных акций Акционерного общества «НИИЭФА им. Д.В. Ефремова» (Санкт-Петербург), размещенных путем закрытой подписки, регистрационный номер дополнительного выпуска: 1-01-05435-D-008D.</td>\n      <td>Решение в отношении ценных бумаг</td>\n      <td>Северо-Западное главное управление Центрального банка Российской Федерации</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>66</th>\n      <td>АО «ИВИ», ОГРН 1197847210604, ИНН 7802697357</td>\n      <td>Банк России 09.04.2021 принял решение о признании дополнительного выпус

In [ ]:
df[df['issuer']=='Сибирское главное управление Центрального банка Российской Федерации'].to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>title</th>\n      <th>desc</th>\n      <th>result</th>\n      <th>issuer</th>\n      <th>decision</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>59</th>\n      <td>АО «НПЗ», ОГРН 1115476008889, ИНН 5402534361</td>\n      <td>Банк России 12.04.2021 принял решение о возобновлении эмиссии и государственной регистрации отчета об итогах дополнительного выпуска обыкновенных акций акционерного общества «Новосибирский приборостроительный завод» (Новосибирская область), размещенных путем закрытой подписки, государственный регистрационный номер дополнительного выпуска 1-01-13009-F-007D.</td>\n      <td>Решение в отношении ценных бумаг</td>\n      <td>Сибирское главное управление Центрального банка Российской Федерации</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>74</th>\n      <td>АО «Абаканское РПП», ОГРН 1031900521135, ИНН 1901057847</td>\n      <td>Банк Росс

In [ ]:
text = input()
word = list()
alphabet='АБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ'
for c in text:
  if c.upper() in alphabet:
    word.apend(c.upper())
  word = ''.join(word)        
print(word)

          

У СУДЬИ ЖАРРИКЕСА ПРОНИЦАТЕЛЬНЫЙ УМ


AttributeError: ignored

In [ ]:
key = int(input())
text = input()
reverse = input()
print(jarriquez_encryption()) 

423
У СУДЬИ ЖАРРИКЕСА ПРОНИЦАТЕЛЬНЫЙ УМ
False


TypeError: ignored